In [15]:
import pandas as pd
import numpy as np

In [16]:
path = "data/"

df = pd.read_csv(path + "train.csv")
df.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [17]:
df_pp = df.copy()

In [18]:
#Scaling

from sklearn.preprocessing import RobustScaler

rb_scaler = RobustScaler()
df_pp.iloc[:, 3:] = rb_scaler.fit_transform(df.iloc[:, 3:])

df_pp.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,0.927836,0.623783,0.753702,0.916931,1.041252,0.639184,0.634486,...,0.838327,0.638332,0.770905,0.825931,0.748549,0.859066,0.664478,0.744135,0.795289,0.978342
1,1,QSO,788,0.223298,0.328228,0.496221,0.713509,0.757221,0.341950,0.386929,...,0.389795,0.412532,0.573234,0.655629,0.823214,0.398328,0.419522,0.539673,0.685895,0.774402
2,2,QSO,427,-1.204186,-1.296932,-1.124607,-1.010056,-0.801392,-1.200267,-1.222701,...,-0.980354,-0.985231,-0.837531,-0.656544,-0.455081,-0.949423,-0.987518,-0.838240,-0.662475,-0.453490
3,3,QSO,864,-0.032482,0.055586,0.191466,0.343934,0.346375,-0.004199,0.099931,...,0.026301,0.140866,0.269585,0.356218,0.426423,0.133448,0.157305,0.250155,0.373217,0.466269
4,4,STAR_RED_DWARF,612,2.226460,1.288028,0.817437,0.198234,-0.090595,1.940455,1.433796,...,2.154141,0.982669,0.836473,0.308756,0.067117,1.639432,1.247397,0.781377,0.282689,0.088207


In [19]:
"""
from sklearn.model_selection import train_test_split

X = df_pp.drop(columns='type')
y = df_pp['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
"""

"\nfrom sklearn.model_selection import train_test_split\n\nX = df_pp.drop(columns='type')\ny = df_pp['type']\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n"

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

X = df_pp.drop(columns='type')
y = df_pp['type']

lgbm = LGBMClassifier(n_estimators=500)
boosting_type = ['gbdt', 'goss', 'dart']
num_leaves = [31,63,127,255]
scale_pos_weight = [1.1, 1.5, 2.0]
max_depth = [5,10,15,20]


parameter = {'num_leaves':num_leaves,
            'boosting_type':boosting_type,
            'scale_pos_weight':scale_pos_weight,
            'max_depth':max_depth}

rs = RandomizedSearchCV(estimator=lgbm,
                    param_distributions=parameter,
                    scoring='neg_log_loss')

rs.fit(X, y)

TypeError: 'set' object does not support indexing

In [13]:
print(rs.best_estimator_)
print(rs.best_score_)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.05, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=400, n_jobs=-1, num_leaves=127, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
-0.43857676209647756
